## Procesamiento y limpieza de Datos

* Se trabaja con los DS suministrados de manera aparte a los fines de generar un DF limpio al cual aplicarle NLP.
* El objetivo buscado es reducir los tiempos de procesamiento, generando un DF fiable trabajado en una notebook aparte.

In [25]:
# Importamos las librerias a utilizar
%config IPCompleter.greedy = True

import pandas as pd
import nltk

import re
from nltk.stem.snowball import SpanishStemmer 

import gc

In [26]:
# Cargamos las 3 bases de datos en sus respectivos DF
data_dev = pd.read_json("dataset_es_dev.json", lines = True)
data_test = pd.read_json("dataset_es_test.json", lines = True)
data_train = pd.read_json("dataset_es_train.json", lines = True)

In [27]:
# Concatenamos los 3 DF anteriores para trabajar con un unico.
df_amazon = pd.concat([data_dev, data_test, data_train])
df_amazon = df_amazon.reset_index(drop=True)

In [28]:
del data_dev
del data_test
del data_train
gc.collect()

247

In [29]:
# Verificamos el shape
df_amazon.shape

(210000, 8)

In [30]:
# Imprimimos el head() a los fines de tener un conocimiento del contenido.
df_amazon.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0417480,product_es_0873923,reviewer_es_0672978,1,"Malisimo, muy grande demasiado aparatoso y mal...",Mala compra,es,wireless
1,es_0180432,product_es_0713146,reviewer_es_0100858,1,No he recibido el pedido no la devolución,No lo he recibido,es,apparel
2,es_0144850,product_es_0356874,reviewer_es_0486447,1,"Tengo que buscar otro sistema, este no funcion...",Que no aprieta bien en el manillar,es,sports
3,es_0339629,product_es_0939832,reviewer_es_0894703,1,Utilicé las brocas de menor diámetro y se me d...,Brocas de mantequilla,es,home_improvement
4,es_0858362,product_es_0489066,reviewer_es_0887663,1,No me gusta su olor a viejo y aspecto malo,No me gusta,es,beauty


In [31]:
df_amazon.groupby(['language']).count()

,review_id,product_id,reviewer_id,stars,review_body,review_title,product_category
language,,,,,,,
es,210000,210000,210000,210000,210000,210000,210000


In [32]:
# Aplicamos un filtro para quedarnos con las columnas que nos interesa trabajar.
df_amazon = df_amazon[['stars', 'review_body', 'review_title', 'product_category']]

In [33]:
df_amazon.head()

,stars,review_body,review_title,product_category
0,1,"Malisimo, muy grande demasiado aparatoso y mal...",Mala compra,wireless
1,1,No he recibido el pedido no la devolución,No lo he recibido,apparel
2,1,"Tengo que buscar otro sistema, este no funcion...",Que no aprieta bien en el manillar,sports
3,1,Utilicé las brocas de menor diámetro y se me d...,Brocas de mantequilla,home_improvement
4,1,No me gusta su olor a viejo y aspecto malo,No me gusta,beauty


In [34]:
# Cargamos las stopwords de nltk en idioma español
stopwords = nltk.corpus.stopwords.words('spanish')

In [35]:
# Palabras a mantener: no, pero, porque, sin, ni, nada
stopwords.remove('no')
stopwords.remove('pero')
stopwords.remove('porque')
stopwords.remove('sin')
stopwords.remove('ni')
stopwords.remove('nada')

In [36]:
stemmer = SpanishStemmer()
reviews_body_stemm=[]
for i in df_amazon.review_body:
   
    reviews_body=re.sub("[^a-zA-Z]"," ",str(i))  # Vamos a reemplzar los caracteres que no sean leras por espacios
    
    reviews_body=reviews_body.lower() # Pasamos todo a minúsculas
    
    reviews_body=nltk.word_tokenize(reviews_body) # Tokenizamos para separar las palabras del titular
    
    reviews_body = [i for i in reviews_body if not i in stopwords] # Sacamos las Stopwords
    
    ## Hasta acá Normalizamos, ahora a stemmizar
    
    reviews_body = [stemmer.stem(word) for word in reviews_body]  # Aplicamos la funcion para buscar la raiz de las palabras
   
    reviews_body = " ".join(reviews_body)  # Por ultimo volvemos a unir el titular
    
    # Vamos armando una lista con todos los titulares
    reviews_body_stemm.append(reviews_body)

In [37]:
reviews_title_stemm=[]
for i in df_amazon.review_title:
   
    reviews_title=re.sub("[^a-zA-Z]"," ",str(i))  # Vamos a reemplzar los caracteres que no sean leras por espacios
    
    reviews_title=reviews_title.lower() # Pasamos todo a minúsculas
    
    reviews_title=nltk.word_tokenize(reviews_title) # Tokenizamos para separar las palabras del titular
    
    reviews_title = [i for i in reviews_title if not i in stopwords] # Sacamos las Stopwords
    
    ## Hasta acá Normalizamos, ahora a stemmizar
    
    reviews_title = [stemmer.stem(word) for word in reviews_title]  # Aplicamos la funcion para buscar la raiz de las palabras
   
    reviews_title = " ".join(reviews_title)  # Por ultimo volvemos a unir el titular
    
    # Vamos armando una lista con todos los titulares
    reviews_title_stemm.append(reviews_title)

In [38]:
# Convertimos las listas en DF
reviews_body_stemm = pd.DataFrame(reviews_body_stemm, columns = ['Body Stemm'])
reviews_title_stemm = pd.DataFrame(reviews_title_stemm, columns = ['Title Stemm'])

In [39]:
# Concatenamos los titulos y el cuerpo stemizados
reviews_stemm = pd.concat([reviews_body_stemm, reviews_title_stemm], axis=1)

In [40]:
# Agregamos los valores al DF de amazon
df_amazon['Stemm_body'] = reviews_stemm['Body Stemm']
df_amazon['Stemm_title'] = reviews_stemm['Title Stemm']
df_amazon.head()

,stars,review_body,review_title,product_category,Stemm_body,Stemm_title
0,1,"Malisimo, muy grande demasiado aparatoso y mal...",Mala compra,wireless,malisim grand demasi aparat mal protector pantall,mal compr
1,1,No he recibido el pedido no la devolución,No lo he recibido,apparel,no recib ped no devoluci n,no recib
2,1,"Tengo que buscar otro sistema, este no funcion...",Que no aprieta bien en el manillar,sports,busc sistem no funcion bien abrazader llev aga...,no apriet bien manill
3,1,Utilicé las brocas de menor diámetro y se me d...,Brocas de mantequilla,home_improvement,utilic broc menor di metr dobl vari si mantequ...,broc mantequill
4,1,No me gusta su olor a viejo y aspecto malo,No me gusta,beauty,no gust olor viej aspect mal,no gust


In [41]:
reviews_body_lemm=[]
for i in df_amazon.review_body:
    # Vamos a reemplzar los caracteres que no sean leras por espacios
    reviews_body = re.sub("[^a-zA-Z]"," ",str(i))
    # Pasamos todo a minúsculas
    reviews_body = reviews_body.lower()
    # Tokenizamos para separar las palabras
    reviews_body = nltk.word_tokenize(reviews_body)
    
    # Sacamos las Stopwords
    reviews_body = [i for i in reviews_body if not i in stopwords]
    
    # Por ultimo volvemos a unir el titular
    reviews_body=" ".join(reviews_body)
    #dataset["titular_normalizado"] = titular_list
    reviews_body_lemm.append(reviews_body)

In [42]:
reviews_title_lemm=[]
for i in df_amazon.review_title:
    # Vamos a reemplzar los caracteres que no sean leras por espacios
    reviews_title = re.sub("[^a-zA-Z]"," ",str(i))
    # Pasamos todo a minúsculas
    reviews_title = reviews_title.lower()
    # Tokenizamos para separar las palabras
    reviews_title = nltk.word_tokenize(reviews_title)
    
    # Sacamos las Stopwords
    reviews_title = [i for i in reviews_title if not i in stopwords]
    
    # Por ultimo volvemos a unir el titular
    reviews_title=" ".join(reviews_title)
    #dataset["titular_normalizado"] = titular_list
    reviews_title_lemm.append(reviews_title)

In [43]:
# Pasamos las dos listas a DF
reviews_body_lemm = pd.DataFrame(reviews_body_lemm, columns = ['Body Lemm'])
reviews_title_lemm = pd.DataFrame(reviews_title_lemm, columns = ['Title Lemm'])

In [44]:
# Generamos un DF con el body y title lemmatizados
reviews_lemm = pd.concat([reviews_body_lemm, reviews_title_lemm], axis=1)

In [45]:
# Agregamos ambas columnas al DF de amazon e imprimimos los 5 primeros registros
df_amazon['Lemm_body'] = reviews_lemm['Body Lemm']
df_amazon['Lemm_title'] = reviews_lemm['Title Lemm']
df_amazon.head()

,stars,review_body,review_title,product_category,Stemm_body,Stemm_title,Lemm_body,Lemm_title
0,1,"Malisimo, muy grande demasiado aparatoso y mal...",Mala compra,wireless,malisim grand demasi aparat mal protector pantall,mal compr,malisimo grande demasiado aparatoso mal protec...,mala compra
1,1,No he recibido el pedido no la devolución,No lo he recibido,apparel,no recib ped no devoluci n,no recib,no recibido pedido no devoluci n,no recibido
2,1,"Tengo que buscar otro sistema, este no funcion...",Que no aprieta bien en el manillar,sports,busc sistem no funcion bien abrazader llev aga...,no apriet bien manill,buscar sistema no funciona bien abrazadera lle...,no aprieta bien manillar
3,1,Utilicé las brocas de menor diámetro y se me d...,Brocas de mantequilla,home_improvement,utilic broc menor di metr dobl vari si mantequ...,broc mantequill,utilic brocas menor di metro doblaron varias s...,brocas mantequilla
4,1,No me gusta su olor a viejo y aspecto malo,No me gusta,beauty,no gust olor viej aspect mal,no gust,no gusta olor viejo aspecto malo,no gusta


In [46]:
df_amazon['Stemm'] = df_amazon['Stemm_body'] + ' ' +df_amazon['Stemm_title']
df_amazon['Lemm'] = df_amazon['Lemm_body'] + ' ' +df_amazon['Lemm_title']
df_amazon.head()

,stars,review_body,review_title,product_category,Stemm_body,Stemm_title,Lemm_body,Lemm_title,Stemm,Lemm
0,1,"Malisimo, muy grande demasiado aparatoso y mal...",Mala compra,wireless,malisim grand demasi aparat mal protector pantall,mal compr,malisimo grande demasiado aparatoso mal protec...,mala compra,malisim grand demasi aparat mal protector pant...,malisimo grande demasiado aparatoso mal protec...
1,1,No he recibido el pedido no la devolución,No lo he recibido,apparel,no recib ped no devoluci n,no recib,no recibido pedido no devoluci n,no recibido,no recib ped no devoluci n no recib,no recibido pedido no devoluci n no recibido
2,1,"Tengo que buscar otro sistema, este no funcion...",Que no aprieta bien en el manillar,sports,busc sistem no funcion bien abrazader llev aga...,no apriet bien manill,buscar sistema no funciona bien abrazadera lle...,no aprieta bien manillar,busc sistem no funcion bien abrazader llev aga...,buscar sistema no funciona bien abrazadera lle...
3,1,Utilicé las brocas de menor diámetro y se me d...,Brocas de mantequilla,home_improvement,utilic broc menor di metr dobl vari si mantequ...,broc mantequill,utilic brocas menor di metro doblaron varias s...,brocas mantequilla,utilic broc menor di metr dobl vari si mantequ...,utilic brocas menor di metro doblaron varias s...
4,1,No me gusta su olor a viejo y aspecto malo,No me gusta,beauty,no gust olor viej aspect mal,no gust,no gusta olor viejo aspecto malo,no gusta,no gust olor viej aspect mal no gust,no gusta olor viejo aspecto malo no gusta


In [47]:
df_amazon = df_amazon.drop(['Stemm_body', 'Stemm_title', 'Lemm_body', 'Lemm_title'], axis = 1)

In [48]:
# Generamos un csv con el DF ya trabajado.
df_amazon.to_csv("amazon.csv")

In [49]:
del df_amazon
del reviews_body_lemm
del reviews_title_lemm
del reviews_lemm
del reviews_body_stemm
del reviews_title_stemm
del reviews_stemm
gc.collect()

22